In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 13})
import pickle
from scipy.optimize import curve_fit
from my_functions import schechter, double_schechter, count_true
from minijpas_LF_and_puricomp import effective_volume

In [ ]:
# My LF

nb_min = 1
nb_max = 25
nbs_list = [[1, 5], [5, 10], [10, 15], [15, 20], [20, 25]]
total_volume = effective_volume(nb_min, nb_max, 'both')

for i, [nb1, nb2] in enumerate(nbs_list):
    pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew15_ewoth400_nb'
    filename = f'{pathname}/LFs.pkl'
    with open(filename, 'rb') as file:
        this_LF_dict = pickle.load(file)

    this_volume = effective_volume(nb1, nb2, 'both')

    # In the first iteration of the loop:
    if i == 0:
        LF_bins = this_LF_dict['LF_bins']
        bin_width = np.diff(LF_bins)[0]

        this_LF_raw = this_LF_dict['LF_total'] * this_volume * bin_width
        LF_raw = this_LF_raw / bin_width
        LF_uncorr = this_LF_dict['LF_total_raw'] * this_volume

        LF_err = ((np.array(this_LF_dict['LF_total_err']) *
                  this_volume * bin_width) ** 2 - this_LF_raw) ** 0.5
    # Second iter and further
    else:
        this_LF_raw = this_LF_dict['LF_total'] * this_volume * bin_width
        LF_raw += this_LF_raw / bin_width
        LF_err += ((np.array(this_LF_dict['LF_total_err']) *
                  this_volume * bin_width) ** 2 - this_LF_raw) ** 0.5
        LF_uncorr += this_LF_dict['LF_total_raw'] * this_volume

LF_err = np.array(LF_err)
LF_err[~np.isfinite(LF_err)] = 0

LF_dict = {
    'LF_bins': LF_bins,
    'LF_total': LF_raw / total_volume,
    'LF_total_uncorr': LF_uncorr / total_volume,
    'LF_total_err': (LF_err ** 2 + LF_raw * bin_width) ** 0.5 / total_volume / bin_width
}

LF_phi = LF_dict['LF_total']
LF_bin = LF_dict['LF_bins']
LF_yerr_minus = LF_dict['LF_total_err'][0]
LF_yerr_plus = LF_dict['LF_total_err'][1]
LF_xerr = np.ones(LF_dict['LF_total_err'][2].shape) * bin_width * 0.5

In [ ]:
def sch_fit(Lx, Phistar, Lstar):
    return schechter(Lx, Phistar, Lstar, -1.5) * Lx * np.log(10)

In [ ]:
where_fit = (LF_bin > 43.5) & (LF_phi > 0)

In [ ]:
survey_list = [f'minijpasAEGIS00{i}' for i in range(1, 5)] + ['jnep']

hist_mat = [
    np.load(f'npy/hist_i_mat_{survey_name}.npy')
    for survey_name in survey_list
]

In [ ]:
N_iter = len(hist_mat[0])

nb_min = 1
nb_max = 25
bin_width = LF_bin[1] - LF_bin[0]
volume = effective_volume(nb_min, nb_max, 'both')

hist_total = np.sum(hist_mat, axis=0)
LF_total = hist_total / bin_width / volume

popt_list = []
N_fits = N_iter
for i in range(N_iter):
    try:
        popt, pcov = curve_fit(sch_fit, 10**LF_bin[where_fit], LF_total[i][where_fit],
                            p0=[1e-6, 1e44])
        popt_list.append(popt)
    except:
        N_fits -= 1
        continue

Phi16, Phi50, Phi84 = np.percentile(popt_list, [16, 50, 84], axis=0)

In [ ]:
%matplotlib inline

In [ ]:
Lx = np.logspace(42, 47)

popt = Phi50

fig, ax = plt.subplots(figsize=(7, 6))

# Plot the reference LF curves

phistar1 = 3.33e-6
Lstar1 = 44.65
alpha1 = -1.35

phistar2 = -3.45
Lstar2 = 42.93
alpha2 = -1.93

Phi_center = double_schechter(
    Lx, phistar1, 10 ** Lstar1, alpha1, 10 ** phistar2, 10 ** Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    zorder=1, color='C6'
)

phistar1 = 10 ** -3.41
Lstar1 = 10 ** 42.87
alpha1 = -1.7

phistar2 = 10 ** -5.85
Lstar2 = 10 ** 44.6
alpha2 = -1.2

Phi_center = double_schechter(
    Lx, phistar1, Lstar1, alpha1, phistar2, Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    zorder=0, color='C7'
)

# Median fit
fit_list = []
for i in range(N_fits):
    Phi_i = sch_fit(Lx, *tuple(popt_list[i]))
    fit_list.append(Phi_i)
Phi = np.percentile(fit_list, 50, axis=0)

ax.plot(np.log10(Lx), Phi, label='My fit')

# Error area
fit_list = []
for i in range(N_fits):
    Phi_i = sch_fit(Lx, *tuple(popt_list[i]))
    fit_list.append(Phi_i)
Phi_minus, Phi_plus = np.percentile(fit_list, [16, 84], axis=0)
ax.fill_between(np.log10(Lx), Phi_plus, Phi_minus, color='gray', alpha=0.2)

ax.errorbar(LF_bin, LF_phi,
            yerr=[LF_yerr_minus, LF_yerr_plus],
            xerr=LF_xerr,
            fmt='s', color='r', capsize=4, label='My points')

ax.set_yscale('log')
ax.set_ylim(1e-8, 5e-3)
ax.set_xlim(42.5, 45.5)
ax.legend()

Phistar = popt[0]
Lstar = np.log10(popt[1])
alpha = -1.5
Phistar_err = (Phi84[0] - Phi16[0]) * 0.5
Lstar_err = (np.log10(Phi84[1]) - np.log10(Phi16[1])) * 0.5

text = fr'''
$\Phi^*=${Phistar:0.1e}$\pm${Phistar_err:0.1e}
$L^*=${Lstar:0.2f}$\pm${Lstar_err:0.2f}
$\alpha=${alpha:0.2f} (fixed)
'''
ax.text(42.7, 1e-7, text)

plt.show()